Adopted from code here: https://colab.research.google.com/github/neural-reckoning/cosyne-tutorial-2022/blob/main/2-coincidence-detection-solution.ipynb 

In [1]:
import ipywidgets as widgets
from brian2 import *
import matplotlib.gridspec as gridspec

prefs.codegen.target = 'numpy'

Input Signal Neurons (sets up the context w/ two ears)-- IPD, ITD, etc. and generates spiking signal based on poisson process

In [2]:
def input_signal(rate_max_Hz=100, ipd_deg=90, f_Hz=3):
    #Assigning units to all inputs
    rate_max = rate_max_Hz*Hz
    ipd = (pi/180) * ipd_deg
    f = f_Hz*Hz
    
    #Defining equations governing ear neurons
    eqs_ears = '''
    theta = 2*pi*f*t + i*ipd : 1
    rate = rate_max*0.5*(1+sin(theta)) : Hz
    '''
    #Group of two neurosn firing according to a Poisson process
    ears = NeuronGroup(2, eqs_ears, threshold='rand()<rate*dt', dt=1*ms)
    
    #Recording the spikes
    M_spike = SpikeMonitor(ears)
    M_state = StateMonitor(ears, 'rate', record=True)
    
    #Running Simulation
    run(1*second)
    
    #Plotting the outputs (ignoring stuff past this)
    trains = M_spike.spike_trains()
    fig = figure(figsize=(4, 2), dpi=200)
    gs = gridspec.GridSpec(2, 1, hspace=0, height_ratios=[1, .3])
    ax = subplot(gs[0])
    plot(M_state.t/ms, M_state.rate[0]/Hz, label='Left ear')
    plot(M_state.t/ms, M_state.rate[1]/Hz, label='Right ear')
    legend(loc='upper right')
    gca().set_frame_on(False)
    ylabel('Rate')
    yticks([])
    xticks([])
    ylim(-10, 210)
    subplot(gs[1], sharex=ax)
    plot(trains[0]/ms, [0]*len(trains[0]), '|')
    plot(trains[1]/ms, [1]*len(trains[1]), '|')
    ylim(-1, 2)
    gca().set_frame_on(False)
    xlabel('Time')
    ylabel('Spikes')
    yticks([])
    xticks([])
    tight_layout()
    
if widgets is not None:
    widgets.interact(input_signal,
        rate_max_Hz=widgets.IntSlider(min=10, max=200, value=100, step=10, continuous_update=False),
        ipd_deg=widgets.IntSlider(min=0, max=360, value=90, step=10, continuous_update=False),
        f_Hz=widgets.FloatSlider(min=0, max=10, value=3, step=.1, continuous_update=False),
        );
else:
    input_signal()

interactive(children=(IntSlider(value=100, continuous_update=False, description='rate_max_Hz', max=200, min=10…

Setting up coincidence detector neurons 

In [4]:
def localize(rate_max_Hz=400, ipd_deg=200, f_Hz=50, w = 0.5, tau_ms=1, N_cd=100, duration=1*second):
    rate_max = rate_max_Hz*Hz
    ipd = (pi/180)*ipd_deg
    f = f_Hz*Hz
    tau = tau_ms*ms
    itd = ipd/(2*pi*f)
    
    #Adding padding on both sides with rate=0.5*rate_max to make sure signal is same on both sides
    eqs_ears = '''
    theta = 2*pi*f*t + i*ipd : 1
    signal_is_on = int(t<duration-itd)*int(i==0)+int(t>itd)*int(i==1) : 1
    rate = rate_max*0.5*(1+signal_is_on*sin(theta)) : Hz
    '''
    
    ears = NeuronGroup(2, eqs_ears, threshold='rand()<rate*dt')
    
    # Stard LIF neuron with added best IPD and best ITD
    
    eqs_cd = '''
    dv/dt = -v/tau : 1
    best_ipd = 2*pi*i/(N_cd-1) : 1
    best_itd = best_ipd/(2*pi*f) : second
    '''
    
    cd = NeuronGroup(N_cd, eqs_cd, threshold='v>1', reset='v=0', method='exact')
    
    #Synapses from ears to coincidence detector neurons
    S = Synapses(ears, cd, on_pre='v += w')
    #All presynaptic neurons are connected to all post synaptic neurons
    S.connect(p=1)
    #Delays are 0 by default so we set the delays for where the presynaptic neuron has index 1
    S.delay['i==1'] = 'best_itd'
    
    M = SpikeMonitor(cd)
    
    run(duration)
    
    #We take as our estimate the mean best IPD of all neurons with maximum spike count
    i = max(M.count)
    I = M.count==i
    ipd_est = mean(cd.best_ipd[I])
    
    figure(figsize=(6, 4), dpi=100)
    plot(cd.best_ipd[I]*180/pi, M.count[I], 'or')
    plot(cd.best_ipd*(180/pi), M.count, '.k')
    axvline(ipd_deg, ls='--', c='b', label='True IPD')
    axvline(ipd_est*180/pi, ls='--', c='r', label='Estimated IPD')
    xlabel('IPD (deg)')
    ylabel('Spike count')
    legend(loc='lower right')
    tight_layout()

if widgets is not None:
    widgets.interact(localize,
        rate_max_Hz=widgets.IntSlider(min=10, max=1000, value=400, step=10, continuous_update=False),
        ipd_deg=widgets.IntSlider(min=0, max=360, value=90, step=10, continuous_update=False),
        f_Hz=widgets.IntSlider(min=0, max=200, value=50, step=5, continuous_update=False),
        w=widgets.FloatSlider(min=.1, max=1, value=.5, step=.1, continuous_update=False),
        tau_ms=widgets.FloatSlider(min=.1, max=10, value=1, step=.1, continuous_update=False),
        N_cd=widgets.IntSlider(min=10, max=1000, value=100, step=10, continuous_update=False),
        duration=widgets.fixed(1*second),
        );
else:
    localize()
    

interactive(children=(IntSlider(value=400, continuous_update=False, description='rate_max_Hz', max=1000, min=1…